In [2]:
import numpy as np

In [109]:
def find_basis(A, tol=1e-3):
    from itertools import combinations
    
    m, n = A.shape
    indexes = range(n)

    for basis in combinations(indexes, m):
        A_b = A[:,basis]
        if np.abs(np.linalg.det(A_b)) > tol:
            basis = set(basis)
            return np.array([i in basis for i in range(n)])
        
def dual_simplex(A, b, c, d_l, d_u, iters=10):
    J_b = find_basis(A)
    m, n = A.shape
    for i in range(1, iters + 1):
        A_b = A[:,J_b]
        B = np.linalg.inv(A_b)

        y = c[J_b].T.dot(B)

        # stage 7
        delta = y.dot(A) - c

        J_h = J_b == False
        J_h_plus = J_h & (delta >= 0)
        J_h_minus = J_h & (J_h_plus == False)

        # stage 2
#         aleph = np.zeros(n)
#         aleph[(J_b == False) & J_h_plus] = d_l[(J_b == False) & J_h_plus]
#         aleph[(J_b == False) & (J_h_plus == False)] = d_u[(J_b == False) & (J_h_plus == False)]
        aleph[J_h_plus] = d_l[J_h_plus]
        aleph[J_h_minus] = d_u[J_h_minus]
        mask = J_h_plus | J_h_minus
        aleph[J_b] = B.dot((b - A[:,mask].dot(aleph[mask])).T)

        # stage 3
        if np.all(d[J_b] <= aleph[J_b]) & np.all(aleph[J_b] <= d_star[J_b]):
            return aleph

        # stage 4
        print(aleph)
        j_k = np.argwhere(((aleph < d_l) | (aleph > d_u)) & J_b).min()
        j_k_idx = np.argwhere(np.argwhere(J_b).ravel() == j_k).min()

        # stage 5
        mu = np.zeros(n)
        mu[j_k] = 1 if aleph[j_k] < d_l[j_k] else -1
        delta_y = mu[j_k] * B[j_k_idx]
        mask = J_b == False
        mask[j_k] = False
        mu[mask] = delta_y.dot(A[:,mask])

        # stage 6
        sigma = np.zeros(n)
        sigma[J_h == False] = np.infty
        mask = J_h & (J_h_plus & (mu < 0)) | (J_h_minus & (mu > 0))
        sigma[mask] = -delta[mask] / mu[mask]
        mask = J_h & ((J_h_plus & (mu < 0)) | (J_h_minus & (mu > 0)) == False)
        sigma[mask] = np.infty

        j_0 = np.argmin(sigma)
        if sigma[j_0] == np.infty:
            raise Exception('There is not solution')

        # stage 8
        J_b[j_k] = False
        J_b[j_0] = True

        J_h = J_b == False
        J_h_plus[j_0] = False

        if mu[j_k] == 1:
            J_h_plus[j_k] = True

        J_h_minus = J_h_plus == False

        J_h = J_b == False

        if mu[j_k] == 1:
            J_h_plus[j_k] = True

        if mu[j_k] in (-1, 1) and J_h_plus[j_0]:
            J_h_plus[j_0] = False

        J_h_minus = J_h & (J_h_plus == False)
    else:
        raise Exception('Did not converge in {} iterations'.format(iters))

## Example 1

In [110]:
A = np.array([
    [2, 1, -1, 0, 0, 1],
    [1, 0, 1, 1, 0, 0],
    [0, 1, 0, 0, 1, 0]
])

b = np.array([2, 5, 0])

c = np.array([3, 2, 0, 3, -2, -4])
d_l = np.array([0, -1, 2, 1, -1, 0])
d_u = np.array([2, 4, 4, 3, 3, 5])

J_b = np.array([False, False, False, True, True, True])

dual_simplex(A, b, c, d_l, d_u)

[ 0.  0.  0.  3. -1.  0.]
[ 0.  0.  2.  0. -1.  0.]


array([ 0.,  0.,  2.,  1., -1.,  0.])

## Problem 1

In [111]:
A = np.array([
    [1, -5, 3, 1, 0, 0],
    [4, -1, 1, 0, 1, 0],
    [2, 4, 2, 0, 0, 1]
], dtype=float)

b = np.array([-7, 22, 30], dtype=float)

c = np.array([7, -2, 6, 0, 5, 2], dtype=float)
d_l = np.array([2, 1, 0, 0, 1, 1], dtype=float)
d_u = np.array([6, 6, 5, 2, 4, 6], dtype=float)

dual_simplex(A, b, c, d_l, d_u)

[ 0.  0.  0.  0.  4.  6.]
[ 2.  0.  0.  0.  4.  6.]
[ 2.  1.  0.  0.  4.  0.]
[ 0.  1.  0.  0.  4.  1.]
[ 2.  1.  0.  0.  4.  0.]
[ 0.  1.  0.  0.  4.  1.]
[ 2.  1.  0.  0.  4.  0.]
[ 0.  1.  0.  0.  4.  1.]
[ 2.  1.  0.  0.  4.  0.]
[ 0.  1.  0.  0.  4.  1.]


Exception: Did not converge in 10 iterations